# MSDM5051 Tutorial 7 - Dynamic Programming

## Content

1. Summary
2. Review & Practice


---
# 1. Summary

Dynamic programming is just the fancy name of **"divide-and-conquer with a table"**. Instead of solving subproblems recursively, we can solve them sequentially and store their solutions in a table , so that whenever the solution to a subproblem is needed, it is already available in the table.

- **Benefit** - You can use dynamic programming whenever a divide-and-conquer method yield an exponential number of subproblems, but only because a small number of subproblems repeated exponentially. It just makes more sense to compute each solution once and store it away in a table for later use, than recomputing it recursively everytime it is needed. 


- **Tradeoff** - Storing the solution of each subproblem takes up computer memory. But it is still a great deal because we have transformed an exponential time problem into a problem that only costs polynomial time and space. Moreover memory is not really a big concern with nowadays' computers.


In the codes in the lecture notes, there are two ways of adding dynamic programming to a recursive function:

- **Create the table by yourself** - Professor has been using a dictionary called `memo` to record the results.

- **Using LRU cache** - LRU means **Least Recently Used**. The LRU cache is basically a *fix-sized queue* structure in the computer memory which stored the most recently called item.  
    - The results of the function is stored one by one to the queue (i.e. first element = oldest, last element = newest).
    - If one result is retrieved, it will be popped out and re-appended at the end of the queue.
    - When the queue is full, the first result (oldest = least recently used) will be deleted so new results can be appended.
    
The Python package `functools` provides the [implementation](https://docs.python.org/3/library/functools.html#functools.lru_cache) of LRU cache as a function decorator. For example:


In [ ]:
from functools import lru_cache

@lru_cache(max_size=None)      # you can limit the size of the queue if you provide a number. Default is 128.
def my_recursive_func(input1, input2, input3):
    
    # ...... whatever this function does
    
    something_returning = my_recursive_func(inputA, inputB, inputC)     # Before running the actual function, it will first search whether
                                                                        # the result from the input combination (inputA, inputB, inputC)
                                                                        # already exist in the LRU cache
    
    # ...... whatever this function does
    
    return something_to_return

---
# 2. Review & Practices

In most examples you can find, understanding the recursive loop is the most difficult part, rather than creating the table. Let's have some review and also pratices on other common textbook problems.

## 2.1. The 0-1 Knapsack problem

The version demonstrated in the lecture is formally called a 0-1 Knapsack problem, i.e. the number of each item $x_i$ can only be 0 or 1. The problem states as follow: Given a bag of capacity $C$ and set of $N$ items with value $\{v_0, v_1,..., v_{N-1}\}$ and weights $\{w_0, w_1,..., v_{N-1}\}$, find the max value (denote as $M$) of items that can fit in the bag with the total weight smaller than the bag's capacity. 

$$
M \ = \max_{x_0,x_1,...,x_{N-1}} \ \sum_i x_i v_i \qquad \text{subject to } \begin{cases}\sum_i x_i w_i \leq C \\ \quad  x_i = 0\text{ or }1\end{cases}
$$

We can see that for the case of $N$ items, there are $2^N$ possible combinations of $\{x_0, x_1,..., x_{N-1}\}$ (although some of them are infeasible solutions that give a total weight $> C$). To construct a recursion, we can think of how these $2^N$ combinations are related the $2^{N-1}$ combinations from the $N-1$ items case, and then $2^{N-2}$ combinations from the $N-2$ items case,... and so on, to the case containing 1 item only. We observe the following rules:

- **Stopping condition** - If capacity $C=0$, max value must be $0$. i.e. 
    $$ M(0, \{x_0, x_1,... x_{N-1}\}) = 0$$

- **Induction step** - Each time an item is added, the capacity is reduced. 
    - If the remaining capacity is smaller than the current item's weight, max value cannot increase further more. i.e. 
        $$ M(C, \{x_0, x_1,... x_{N-1}\}) = M(C, \{x_0, x_1,... x_{N-2}\}) \quad \text{if }C<w_N$$

    - If the remaining capacity is larger than the item's weight, we can check whether we can get a new max value from 
        - a case with the current item added - for this to be valid, this relation must hold:
            $$M(C, \{x_0, x_1,... x_{N-1}\}) - v_{N-1} = M(C-w_{N-1}, \{x_0, x_1,... x_{N-2}\}) \text{ with }C-w_{N-1} \geq 0$$
            i.e. We should be able to find some valid combinations from the previous $N-1$ items that make up a weight less than $C-w_N$. Or else it leads to contradiction.<br><br>
        
        - a case which the current item is not added, but there are some combinations from the previous $N-1$ items that already give a high value (and they already filled the capacity so that $x_N$ cannot be added to that combination to give an even higher value).  
            $$M(C, \{x_0, x_1,... x_{N-1}\}) = M(C, \{x_0, x_1,... x_{N-2}\})$$
        
These rules lead to the standard algorithm of 0-1 knapsack problem:

In [ ]:
# let total item be N
# let value_list = [v0, v2, v3,..., v_N-1] = value of items be a global list
# let weight_list = [w0, w2, w3,..., w_N-1] = weight of items be a global list

def knapsack(capacity, N=len(value_list)):

    if capacity<=0:
        return 0

    if capacity < weight_list[N-1]:
        return knapsack(capacity, N-1)

    return max(knapsack(capacity-weight_list[N-1], N-1) + value_list[N-1],
               knapsack(capacity, N-1))
    

# DIY to make it dynamic programming!

## 2.2. Rod cutting

The rod cutting problem is described as follow - Given a rod of length $L$ and a table of prices $p_l$ for rods of length $l$ ($L$ and $l$ being integers), determine the maximum profit $P$ obtainable by cutting up the rod and selling the pieces. 

We can see some similarity to the knapsack problem in how a solution can look like - in a rod of length $L$, there are $L-1$ positions $\{x_1,x_2,...,x_{L-1}\}$ where we can decide whether to make a cut ($=0$ or $1$) - which gives us $2^{L-1}$ possible combinations. The main differences are the rules of how solutions are related to each other:

- **Stopping condition** - If length of rod $L=1$, max value must be $p_1$. i.e. 
    $$P(1, \{\text{no cut}\}) = p_1$$

- **Induction step** - When the rod has a remaining length of $L$, we can decide whether to make a cut at *EACH* of the $L-1$ position to see if the profit can increase. If we decide to make a cut at position $x_l$ (and no further cut on this length $l$ piece), this relation must hold:
    $$P(L, \{x_1, x_2,... x_{L-1}\}) - p_l = P(L-l, \{x_1, x_2,... x_{L-1-l}\})$$

These rules lead to the standard algorithm of rod cutting problem:


In [ ]:
# let price_table be a global variable = [0, p1, p2, ...p_L]
# so that price_table[i] = price of a piece of length i

def cut_rod(L):
    
    if L<=0:
        return 0
    
    profit = 0            
    for i in range(1, L+1):             # compare with cases where one of the position is cut
                                        
        profit = max(profit, cut_rod(L-i) + price_table[i])     # Note that the case i=L means nowhere is cut, 
                                                                # and will return cut_rod(0) + price_table[L] = 0 + p_L
        
    return profit

# DIY to make it dynamic programming!

## 2.3. Exercise: Matrix chain multiplication

When a matrix of dimension $m\times p$ is multiplied to another matrix of dimension $p\times n$, a total of $m\times p\times n$ multiplication is required. When we have a chain of matrix multiplying, we want to find an order that gives the minimum no. of multiplication that we need to perform. For example, given 3 matrices of dimension $\dim(A_1) = 10\times 100, \dim(A_2) = 100\times 5$ and $\dim(A_3) = 5\times 50$,

- Multiplying $A_1\times A_2$ first, then with $A_3$, i.e. $(A_1A_2)A_3$
    - $(A_1A_2)$ needs $10\times 100\times 5 = 5000$ multiplications. Dimension of resultant matrix = $10\times 5$.
    - $(...)A_3$ needs $10\times 5\times 50=2500$ multiplications.
    - Total $=5000+2500=7500$ multiplications.


- Multiplying $A_2\times A_3$ first, then with $A_1$, i.e. $A_1(A_2A_3)$
    - $(A_2A_3)$ needs $100\times 5\times 50 = 25000$ multiplications. Dimension of resultant matrix = $100\times 50$.
    - $A_1(...)$ needs $10\times 100\times 50=50000$ multiplications.
    - Total $=25000+50000=75000$ multiplications.
    
So different multiplication order makes a huge difference.
 
**Idea:** Given a chain of matrices $A_1A_2...A_N$, we can start by spliting the chain into two, e.g. chain 1 being $A_1A_2...A_k$ and chain 2 being $A_{k+1}...A_N$, such that all multiplication on both chains are finished before the products of the two chains multiplying together. And then we choose the $k$ that costs the minimal no. of multiplication. 

Let the dimension of the $k$th matrix be $\dim(A_k) = m_k\times n_k$ (and we require $n_k =m_{k+1}$ for valid multiplication). Also let $P(A_1A_2...A_N)$ be the total of no. of required multiplication of the whole chain. Try to write out the recursive definition and then the program. 

In [ ]:
# let dim be a list recording the matrices' dimension such that
# dim[0] = m1
# dim[1] = n1 = m2
# ...
# dim[N-1] = n_N-1 = m_N
# dim[N] = n_N


def matmulnum(dim):
    # try it yourself!